# Equal Weighted Like Portfolio - comparisons

This example compares the performance of an equal weighted portfolio relative to several risk-based optimal portfolios with similar risk profile. 

The CVaR, SMCR, MAD, LSSD, BTAD, BTSD, GINI, SD and MV risk-based portfolios are considered. We use the `rtype=InvNrisk` optimization flag (the optimal portfolio with the same risk as the equal weights portfolio).

The results are presented below. We concluded that, at least for the data in this example, there is no clear-cut preference for a portfolio construction (with a superior performance under all market conditions).


We start by importing **azapy** and other useful packages.

In [29]:
import sys
sys.path.append("..")
import azapy as az

print(f"azapy version {az.version()}")

azapy version 1.2.0


### Collect historical market data

- `symb` is the list of stock symbols (portfolio components).
- `sdate` and `edate` are the start and end dates of historical time-series.
- `mktdir` is the name of the directory used as a buffer for market data collected from the data provider (in this case _alphavantage_).
    
> Note: if the flag `force=False` then a reading from `dir=mktdir` is attempted. If it fails, then the data provider servers will be accessed. The new data will be saved to the `dir=mktdir`. For more information see the readMkT documentation https://azapy.readthedocs.io/en/latest/.

In [30]:
symb = ['GLD', 'TLT', 'XLV', 'VGT', 'PSJ']

sdate = "2012-01-01"
edate = "2021-07-27"

mktdir = "../MkTdata"

mktdata = az.readMkT(symb, sdate=sdate, edate=edate, file_dir=mktdir)

read GLD data from file
read TLT data from file
read XLV data from file
read VGT data from file
read PSJ data from file

Request between 2012-01-01 : 2021-07-27
                    GLD         TLT         XLV         VGT         PSJ
source            yahoo       yahoo       yahoo       yahoo       yahoo
force             False       False       False       False       False
save               True        True        True        True        True
file_dir     ../MkTdata  ../MkTdata  ../MkTdata  ../MkTdata  ../MkTdata
file_format         csv         csv         csv         csv         csv
api_key            None        None        None        None        None
verbose            True        True        True        True        True
error                No          No          No          No          No
nrow               2407        2407        2407        2407        2407
sdate        2012-01-03  2012-01-03  2012-01-03  2012-01-03  2012-01-03
edate        2021-07-27  2021-07-27  2021-07-27

### Set scenarios model parameters

Since we plan to run many scenarios, it is useful to organize their model parameters in a dictionary.
The keys are the portfolio names, and the values are the model parameters organized also as dictionaries:
- 'type' : is the portfolio class used in the simulation,
- 'm_param' : is a dictionary with parameters for `set_model` functions.

Note that:
- `'P_N'` is the reference equal weighted portfolio,
- `'P_CVaR'` is the mCVaR optimal portfolio with confidence levels `alpha_CVaR` equal weighted,
- `'P_SMCR'` is the mSMCR optimal portfolio with confidence levels `alpha_SMCR` equal weighted,
- `'P_EVaR'` is the mEVaR optimal portfolio with confidence levels `alpha_EVaR` equal weighted,
- `'P_MAD'` is the mMAD optimal portfolio with weights `coef_MAD`,
- `'P_LSD'` is the mLSD optimal portfolio with weights `coef_LSD`,
- `'P_BTAD'` is the mBTAD optimal portfolio with threshold levels `alpha_BTAD` equal weighted,
- `'P_BTSD'` is the mBTSD optimal portfolio with threshold levels `alpha_BTSD` equal weighted,
- `'P_MV'` is the MV (mean-variance) optimal portfolio,
- `'P_SD'` is the SD optimal portfolio.

- In all cases the optimization type is set to `rtype='InvNRisk'` (optimal portfolio with the same risk profile as the equal weighted portfolio)
- The `'hlength'` parameter (the length of historical data used in weights calibration) is set to 3.25 years except for `'P_GINI'` and `P_SMGINI` where it is set to 1.25 years (for computational speed convenience).

> In this case P_MV and P_SD are the same. 

In [31]:
rtype = 'InvNrisk'
alpha_CVaR = [0.95, 0.90, 0.85]
alpha_SMCR = [0.9, 0.80]
alpha_EVaR = [0.65, 0.6]
coef_MAD = [1./3.] * 3
coef_LSD = [1./3.] * 3
alpha_BTAD = [-0.01, 0]
alpha_BTSD = [-0.01, 0]
hlength = 1.25
verbose = False

models = {'P_CVaR': {'type': 'Port_CVaR', 'm_param': {'alpha': alpha_CVaR, 'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          'P_SMCR': {'type': 'Port_SMCR', 'm_param': {'alpha': alpha_SMCR, 'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          'P_EVaR': {'type': 'Port_EVaR', 'm_param': {'alpha': alpha_EVaR, 'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          'P_MAD': {'type': 'Port_MAD', 'm_param': {'coef': coef_MAD, 'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          'P_LSD': {'type': 'Port_LSD', 'm_param': {'coef': coef_LSD, 'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          'P_BTAD': {'type': 'Port_BTAD', 'm_param': {'alpha': alpha_BTAD, 'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          'P_BTSD': {'type': 'Port_BTSD', 'm_param': {'alpha': alpha_BTSD, 'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          'P_GINI': {'type': 'Port_GINI', 'm_param': {'rtype': rtype, 'hlength': 1.25, 'verbose': verbose}},
          'P_MV': {'type': 'Port_MV', 'm_param': {'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          'P_SD': {'type': 'Port_SD', 'm_param': {'rtype': rtype, 'hlength': hlength, 'verbose': verbose}},
          'P_N': {'type': 'Port_ConstW', 'm_param': {'ww': None}}}

### Main computation loop

- `port` is a list containing the simulated time-series. We will use it to setup a simple portfolio *(see the documentation for `Port_Simple` class)*. It is a very convenient way to facilitate the visual and numerical comparisons between these portfolio performances. 
- `pp` is a dictionary holding the portfolio objects. They may be used later for further analytical inquires.  

In [32]:
port = []
pp = {}
for key, val in models.items():
    ppz = getattr(az, val['type'])
    pp_ = ppz(mktdata, pname=key)
    pp[key] = pp_
    port_ = pp_.set_model(**val['m_param'])
    port.append(port_)

In [33]:
pp['P_GINI'].pname

'P_GINI'

### Prepare the results for comparisons 

Build a `Port_Simple` class holding all the computed portfolios as components. We use this structure to build comparative graphical and numerical performance reports. The aggregated portfolio of portfolios time-series will be neglected.

>Note the call to `set_model` method that is a must.

>Observation: `Port_Simple` is the class that supports the back testing of "Buy and Hold" portfolio (_see its documentation_).
It also can be used as a tool to compare the performance of multiple portfolios. Here we use it in this latter capacity.

In [34]:
ps = az.Port_Simple(port, col='close', pname='ALL')
_ = ps.set_model()

### Time-series visualization 

We used the flag `componly=True` to plot only the portfolio components. 

In [35]:
_ = ps.port_view_all(componly=True, fancy=True, title="Relative performance")

### Portfolio performances

Note that, at least for the duration of this simulation, the P_BTAD portfolio has a similar rate of returns as 
P_N. However, P_BTAD has higher maximum drawdown than P_N. P_N has the smallest maximum drawdown among all portfolios. 

> Note: in this case P_MV anf P_SD portfolios are the same.

In [36]:
ps.port_perf(componly=True, fancy=True)

,RR,DD,RoMaD,DD_date,DD_start,DD_end
symbol,,,,,,
P_N,14.23,-17.44,0.816117,2020-03-18,2020-02-19,2020-04-24
P_EVaR,13.00,-16.46,0.789529,2016-12-15,2016-09-07,2017-06-02
P_CVaR,13.22,-16.98,0.778811,2018-12-24,2018-09-14,2019-07-03
P_LSD,12.91,-16.61,0.777470,2018-12-24,2018-09-14,2019-07-03
P_BTSD,12.89,-16.87,0.764053,2016-12-15,2016-07-08,2017-06-02
P_SMCR,13.54,-17.88,0.757415,2018-12-24,2018-09-14,2019-08-01
P_MAD,12.95,-18.30,0.707370,2016-12-14,2016-07-08,2017-08-31
P_MV,12.97,-18.65,0.695347,2016-12-14,2016-07-08,2017-09-11
P_SD,12.97,-18.65,0.695347,2016-12-14,2016-07-08,2017-09-11


### Annual returns

Remarks:

- 2015 (only half year in this simulation) and 2016, P_N is the best performer.
- 2016 P_N is the best while P_GINI is the worst.
- 2017 P_N is the worst performer. The best performers are P_GINI.
- 2018 P_GINI has the best performance. P_N is second.
- 2019 The best performers are P_BTAD closely follow by P_CVaR, P_MAD and P_LSD. P_GINI and P_N are the worst.
- 2020 The best performers are P_GINI follow by P_N.
- 2021 (first half of the year in this simulation) P_GINI is the leader followed by P_N and P_MV (P_SD is the same with P_MV).

Conclusion: Depending on the historical period P_N may be an outperformer or an underperformer. Moreover, similar remarks can be made about any other portfolio strategy.

> Note: the flags `withcomp=True` includes the portfolios components while the flag `componly=True` excludes the aggregated portfolio of portfolios.


In [37]:
ps.port_annual_returns(withcomp=True, componly=True, fancy=True)

symbol,P_BTAD,P_BTSD,P_CVaR,P_EVaR,P_GINI,P_LSD,P_MAD,P_MV,P_N,P_SD,P_SMCR
year,,,,,,,,,,,
2015,-5.95%,-7.90%,-6.24%,-6.59%,-5.77%,-7.04%,-7.16%,-6.14%,-1.67%,-6.14%,-3.81%
2016,-5.59%,-3.48%,-3.84%,-3.72%,-5.65%,-3.91%,-5.20%,-5.30%,8.28%,-5.30%,-4.70%
2017,36.02%,35.20%,34.26%,34.57%,35.39%,34.51%,36.04%,35.21%,23.10%,35.21%,35.74%
2018,3.77%,1.57%,1.36%,0.76%,3.37%,0.94%,4.55%,5.08%,2.60%,5.08%,-0.92%
2019,20.20%,20.07%,21.65%,21.45%,20.38%,21.16%,19.59%,19.88%,25.81%,19.88%,22.87%
2020,29.73%,34.02%,34.86%,33.45%,30.95%,34.38%,31.53%,31.12%,25.40%,31.12%,32.12%
2021,5.14%,5.89%,5.99%,6.11%,4.99%,5.93%,5.88%,5.06%,4.90%,5.06%,8.19%


### Monthly returns

> Note: the flags `withcomp=True` includes the portfolios components while the flag `componly=True` excludes the aggregated portfolio. 

In [38]:
ps.port_monthly_returns(withcomp=True, componly=True, fancy=True)

### A closer look at P_BTAD portfolio

Monthly returns: are volatile.

In [39]:
pp['P_BTAD'].port_monthly_returns(fancy=True)

year,2015,2016,2017,2018,2019,2020,2021
month,,,,,,,
1,nan%,-0.51%,3.77%,6.39%,8.42%,4.20%,-0.69%
2,nan%,3.37%,4.02%,-0.15%,2.04%,-0.82%,-2.65%
3,nan%,2.57%,0.47%,-0.89%,-2.65%,-2.95%,-4.53%
4,nan%,-0.92%,2.64%,1.65%,1.40%,11.65%,1.41%
5,nan%,2.50%,3.48%,2.67%,-1.32%,5.93%,-0.98%
6,-2.26%,3.53%,-3.96%,-2.76%,7.38%,0.73%,4.67%
7,1.84%,1.10%,4.98%,-0.22%,0.93%,4.84%,3.03%
8,-7.81%,0.20%,2.56%,6.01%,2.39%,-1.92%,nan%
9,-3.21%,-3.25%,2.54%,-0.68%,-3.58%,0.31%,nan%


### P_BTAD performance

An example of individual portfolio performance inquiry. 

In [40]:
pp['P_BTAD'].port_perf(fancy=True)

,RR,DD,RoMaD,DD_date,DD_start,DD_end
symbol,,,,,,
P_BTAD,12.71,-19.18,0.662706,2016-12-14,2016-07-08,2017-09-12
VGT,23.10,-31.84,0.725512,2020-03-23,2020-02-19,2020-06-09
PSJ,22.12,-30.69,0.720991,2020-03-16,2020-02-19,2020-05-22
XLV,16.74,-28.40,0.589180,2020-03-23,2020-01-22,2020-07-15
TLT,4.89,-21.34,0.229103,2021-03-18,2020-08-04,NaN
GLD,0.81,-42.11,0.019280,2015-12-17,2012-10-04,2020-07-22


### P_BTAD drawdowns

Remarks: The first 2 largest drawdowns are similar. However, the 2018 drawdown was caused by the unexpected Feds rate increase while the 2020 event is the Covid-19 pandemic crash. 

In [41]:
pp['P_BTAD'].port_drawdown(fancy=True)

,DD,Date,Start,End
No,,,,
1,-19.18,2016-12-14,2016-07-08,2017-09-12
2,-15.54,2015-09-28,2015-07-20,2016-06-01
3,-15.21,2020-03-18,2020-03-06,2020-04-14
4,-12.81,2018-12-24,2018-09-13,2019-02-14
5,-9.67,2021-05-13,2021-02-11,2021-07-07


### P_BTAD weights

Remarks: At present the P_BTAD portfolio is dominated by GLD and VGT.

In [42]:
pp['P_BTAD'].get_weights(fancy=True)

,Droll,Dfix,GLD,TLT,XLV,VGT,PSJ,_CASH_
0,2015-06-25,2015-06-24,0.41,0.00,95.62,0.00,3.97,0.0
1,2015-09-25,2015-09-24,17.81,6.63,71.07,4.49,0.00,0.0
2,2015-12-28,2015-12-24,22.60,23.59,0.00,17.92,35.89,0.0
3,2016-03-28,2016-03-24,0.00,62.18,0.00,0.00,37.82,0.0
4,2016-06-27,2016-06-24,0.00,92.77,0.00,0.00,7.23,0.0
5,2016-09-27,2016-09-26,22.17,77.83,0.00,0.00,0.00,0.0
6,2016-12-27,2016-12-23,24.09,0.00,0.00,75.91,0.00,0.0
7,2017-03-28,2017-03-27,0.00,0.00,0.00,100.00,0.00,0.0
8,2017-06-27,2017-06-26,0.00,0.00,0.00,100.00,0.00,0.0
9,2017-09-26,2017-09-25,0.00,0.00,0.00,100.00,0.00,0.0
